# **Interactive Location Graph with Google Maps API and GPT-4**

This project generates an interactive location graph using the Google Maps API, OpenAI's GPT-4, and Dash. The graph visualizes nearby places with enriched information and enables dynamic selection of starting points to recalculate distances.

---

## **Steps Overview**

### **Step 1: Fetch Nearby Places**
- **Function:** `get_nearby_places(lat, lon, api_key, radius=10000, place_type=None, keyword=None)`
- **Description:** Queries the Google Maps API to find places near a given latitude and longitude. Filters can be applied using `place_type` and `keyword`.
- **Inputs:**
  - `lat`, `lon`: Latitude and longitude of the user's location.
  - `radius`: Search radius in meters (default: 10,000 meters).
  - `place_type`: Optional, filters results by type (e.g., "restaurant").
  - `keyword`: Optional, refines search results using a specific keyword.
- **Output:** A dictionary of results from the Google Maps API.

---

### **Step 2: Fetch Full Address Details**
- **Function:** `get_place_details(place_id, api_key)`
- **Description:** Uses the Google Place Details API to retrieve the full address for a specific place ID.
- **Inputs:**
  - `place_id`: Unique ID for the place.
  - `api_key`: Google Maps API key.
- **Output:** The formatted address of the place.

---

### **Step 3: Enrich Data with GPT-4**
- **Function:** `analyze_places_with_gpt(places_data, api_key, gpt_api_key, user_lat, user_lon)`
- **Description:** Processes place data through OpenAI GPT-4 to generate concise summaries of each place. Adds details like distance and enriched metadata.
- **Inputs:**
  - `places_data`: The results from the Google Maps API.
  - `api_key`: Google Maps API key.
  - `gpt_api_key`: OpenAI API key.
  - `user_lat`, `user_lon`: User's latitude and longitude.
- **Outputs:**
  - `nodes`: Places with details like name, group, address, distance, and GPT-generated summary.
  - `edges`: Connections between nodes with distances in miles.

---

### **Step 4: Create the Graph**
- **Function:** `create_graph(nodes, edges, lat, lon)`
- **Description:** Creates a network graph using `networkx` and `plotly`. Nodes represent places, and edges represent distances between them.
- **Inputs:**
  - `nodes`: List of place nodes with attributes like name, group, address, and distance.
  - `edges`: List of connections between nodes with distances as labels.
  - `lat`, `lon`: Latitude and longitude to identify the starting node.
- **Output:** An interactive `plotly` figure visualizing the graph.

---

### **Step 5: Run the Interactive App**
- **Function:** `run_interactive_app(nodes, edges, lat, lon)`
- **Description:** Creates a Dash web application to visualize the graph. Allows users to select a starting point dynamically, recalculating distances in real-time.
- **Features:**
  - Dropdown menu to select the starting point.
  - Interactive graph with hover information on nodes and edges.
- **Inputs:**
  - `nodes`: List of enriched nodes with details and addresses.
  - `edges`: List of edges connecting the nodes.
  - `lat`, `lon`: User's latitude and longitude to set the initial starting point.

---

## **How It Works**
1. Fetch nearby places using Google Maps API.
2. Retrieve full addresses for each place using the Place Details API.
3. Enrich place data with summaries from GPT-4.
4. Calculate distances between all locations using the Haversine formula.
5. Create an interactive graph with `plotly` and `networkx`.
6. Launch a Dash application to allow dynamic exploration of the graph.

---

## **Key Features**
- Dynamic starting point selection with real-time distance recalculations.
- GPT-generated summaries for each location.
- Interactive graph visualization with hoverable nodes and edges.

---

## **Requirements**
- Python libraries:
  - `requests`
  - `networkx`
  - `plotly`
  - `dash`
  - `dash-bootstrap-components`
- APIs:
  - Google Maps API
  - OpenAI GPT-4 API

---

## **Sample Usage**

```python
# Replace with your API keys
google_api_key = "YOUR_GOOGLE_API_KEY"
gpt_api_key = "YOUR_OPENAI_API_KEY"

# Example user location
latitude, longitude = 37.7749, -122.4194  # San Francisco

# Fetch nearby places
places_data = get_nearby_places(latitude, longitude, google_api_key, radius=5000, place_type="restaurant")

# Enrich places with GPT-4 and calculate distances
nodes, edges = analyze_places_with_gpt(places_data, google_api_key, gpt_api_key, latitude, longitude)

# Launch the interactive app
run_interactive_app(nodes, edges, latitude, longitude)


In [2]:
import requests
import networkx as nx
import plotly.graph_objects as go
import plotly.graph_objects as go
from math import radians, cos, sin, sqrt, atan2

In [1]:
!pip install openai

In [3]:
pip install numpy


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:

# Step 1: Fetch device's latitude and longitude
def get_device_coordinates():
    try:
        response = requests.get("http://ip-api.com/json/")
        if response.status_code == 200:
            data = response.json()
            return data["lat"], data["lon"]
        else:
            raise Exception(f"Error fetching device location: {response.text}")
    except Exception as e:
        print(f"Failed to fetch device coordinates: {e}")
        return None, None


In [7]:

# # Step 2: Query Google Maps API for nearest outlets

def get_nearby_places(lat, lon, api_key, radius=10000, place_type=None, keyword=None):
    """
    Queries Google Maps API to fetch nearby places with refined parameters.

    Args:
        lat (float): Latitude of the location.
        lon (float): Longitude of the location.
        api_key (str): Google Maps API key.
        radius (int): Search radius in meters (default: 1000).
        place_type (str): Optional type of place to filter (e.g., 'restaurant', 'gas_station').
        keyword (str): Optional keyword to refine search (e.g., 'pizza', 'coffee').

    Returns:
        dict: JSON response from the Google Maps API.
    """
    print(f"Searching places near Latitude: {lat}, Longitude: {lon} within {radius} meters")
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lon}",
        "radius": radius,
        "key": api_key,
        "language": "en"
    }

    if place_type:
        params["type"] = place_type  # Filter by place type
    if keyword:
        params["keyword"] = keyword  # Refine search by keyword

    response = requests.get(url, params=params)

    # Debug response to identify issues
    if response.status_code != 200:
        raise Exception(f"Google Maps API error: {response.text}")
    
    data = response.json()

    # Debugging: Print raw data for verification
    if "results" in data:
        print(f"Found {len(data['results'])} places.")
    else:
        print("No places found.")

    if "results" not in data or len(data["results"]) == 0:
        raise ValueError("No places found for the given parameters. Try increasing the radius or refining your filters.")
    
    return data

# Step 3: Fetch full address for each place using Place Details API
def get_place_details(place_id, api_key):
    url = f"https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        "place_id": place_id,
        "fields": "formatted_address",
        "key": api_key
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Google Maps Place Details API error: {response.text}")
    data = response.json()
    return data.get("result", {}).get("formatted_address", "Address not available")


# Step 4: Calculate distances between two coordinates using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.8  # Earth radius in miles
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c




In [8]:

from openai import OpenAI  # For GPT-4 API

def analyze_places_with_gpt(places_data, api_key, gpt_api_key, user_lat, user_lon):
    openai_client = OpenAI(api_key=gpt_api_key)
    nodes = []
    edges = []

    for i, place in enumerate(places_data.get("results", [])):
        # Basic place information
        place_id = place.get("place_id")
        place_name = place.get("name", "Unknown")
        place_types = ", ".join(place.get("types", []))
        lat = place["geometry"]["location"]["lat"]
        lon = place["geometry"]["location"]["lng"]

        # Fetch full address
        full_address = get_place_details(place_id, api_key)

        # Prepare GPT input
        gpt_prompt = f"""
        You are an AI assistant. Based on the following information, provide a concise summary in English:
        Name: {place_name}
        Types: {place_types}
        Address: {full_address}
        Latitude: {lat}
        Longitude: {lon}
        """
        try:
            # Get GPT-generated details
            openai_client=OpenAI(api_key=gpt_api_key)
            # Get GPT-generated details
            gpt_response = openai_client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are an AI assistant providing concise summaries for locations."},
                    {"role": "user", "content": gpt_prompt}
                ]
            )
            gpt_output = gpt_response.choices[0].message.content.strip()
        except Exception as e:
            gpt_output = f"Failed to retrieve GPT details: {e}"

        # Limit text length
        if len(gpt_output) > 200:
            gpt_output = gpt_output[:200] + "..."

        # Calculate distance from user's location
        distance = haversine(user_lat, user_lon, lat, lon)

        # Add node with GPT details and address
        nodes.append({
            "id": str(i),
            "label": place_name,
            "group": place_types.split(",")[0] if place_types else "Other",
            "lat": lat,
            "lon": lon,
            "details": gpt_output,
            "address": full_address,
            "distance": round(distance, 2)  # Round to 2 decimal places
        })

    # Create edges with distances between nodes
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            dist = haversine(nodes[i]["lat"], nodes[i]["lon"], nodes[j]["lat"], nodes[j]["lon"])
            edges.append({
                "source": nodes[i]["id"],
                "target": nodes[j]["id"],
                "label": f"{round(dist, 2)} miles"
            })

    return nodes, edges


In [ ]:
!pip install dash 


In [ ]:
!pip install dash-bootstrap-components 

In [12]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc

In [13]:
def create_graph(nodes, edges, lat, lon):
    # Determine the starting node based on latitude and longitude
    starting_point_id = None
    min_distance = float("inf")

    for node in nodes:
        # Calculate distance between provided lat/lon and node's lat/lon
        node_lat = node.get("lat", 0)
        node_lon = node.get("lon", 0)
        distance = haversine(lat, lon, node_lat, node_lon)

        # Identify the closest node as the starting point
        if distance < min_distance:
            min_distance = distance
            starting_point_id = node["id"]

    graph = nx.DiGraph()

    # Add nodes
    for node in nodes:
        graph.add_node(node["id"],
                       label=node["label"],
                       address=node["address"],
                       distance=node["distance"],
                       details=node["details"],
                       is_start=(node["id"] == starting_point_id))

    # Add edges
    for edge in edges:
        graph.add_edge(edge["source"], edge["target"], label=edge["label"])

    # Compute positions
    pos = nx.spring_layout(graph)

    # Prepare edge traces
    edge_x, edge_y, edge_hover_texts = [], [], []
    for edge in graph.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_hover_texts.append(f"Distance: {edge[2]['label']} miles")

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color="#888"),
        hoverinfo='text',
        hovertext=edge_hover_texts,
        mode='lines'
    )

    # Prepare node traces
    node_x, node_y, hover_texts, node_names, node_colors = [], [], [], [], []
    for node_id, node_data in graph.nodes(data=True):
        x, y = pos[node_id]
        node_x.append(x)
        node_y.append(y)
        hover_texts.append(
            f"<b>{node_data['label']}</b><br>"
            f"Address: {node_data['address']}<br>"
            f"Distance: {node_data['distance']} miles<br>"
            f"Details: {node_data['details'][:50]}..."  # Shortened details for hover text
        )
        node_names.append(node_data["label"])
        node_colors.append("red" if node_data["is_start"] else "blue")

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=node_names,
        textposition="top center",
        hoverinfo='text',
        hovertext=hover_texts,
        marker=dict(size=15, color=node_colors, line=dict(width=1, color="black"))
    )

    # Responsive layout
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="Interactive Location Graph",
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=40, l=40, r=40, t=30),
                        height=500,  # Adjust height for responsiveness
                        autosize=True
                    ))

    return fig




In [14]:
def run_interactive_app(nodes, edges, lat, lon):
    app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

    # Identify the starting point dynamically
    starting_point_id = None
    min_distance = float("inf")

    for node in nodes:
        distance = haversine(lat, lon, node["lat"], node["lon"])
        node["distance"] = round(distance, 2)  # Update distances
        if distance < min_distance:
            min_distance = distance
            starting_point_id = node["id"]

    app.layout = dbc.Container([
        html.H1("Interactive Location Graph", className="text-center mt-4"),
        html.Div([
            html.Label("Select a Starting Point:"),
            dcc.Dropdown(
                id="node-dropdown",
                options=[{"label": node["label"], "value": node["id"]} for node in nodes],
                value=starting_point_id,
                clearable=False
            ),
        ], className="mb-4"),
        html.Div(
            dcc.Graph(id="location-graph", config={"displayModeBar": False}),
            style={"width": "100%", "height": "auto"}  # Ensure the container is flexible
        )
    ])

    @app.callback(
        Output("location-graph", "figure"),
        Input("node-dropdown", "value")
    )
    def update_graph(new_starting_point_id):
        # Update distances based on the new starting point
        start_node = next(n for n in nodes if n["id"] == new_starting_point_id)
        for node in nodes:
            if node["id"] == new_starting_point_id:
                node["distance"] = 0
            else:
                node["distance"] = round(haversine(start_node["lat"], start_node["lon"], node["lat"], node["lon"]), 2)

        # Recreate graph with updated distances
        return create_graph(nodes, edges, start_node["lat"], start_node["lon"])

    app.run_server(debug=True)


In [16]:
# Main Workflow
def main(google_api_key, gpt_api_key):
    print("Fetching device coordinates...")
    lat, lon = get_device_coordinates()
    if lat is None or lon is None:
        print("Could not determine device location.")
        return

    print(f"Device coordinates: Latitude = {lat}, Longitude = {lon}")
    print("Finding nearby places...")
    places_data = get_nearby_places(lat, lon, google_api_key, 5000, 'restaurant')
    print("Analyzing places with GPT-4o and adding full addresses...")
    nodes, edges = analyze_places_with_gpt(places_data, google_api_key, gpt_api_key, lat, lon)
    print("Launching interactive graph app...")
    run_interactive_app(nodes, edges, lat, lon)



In [17]:
google_maps_api_key = "" # google map api key
gpt_api_key = "" #openai key

main(google_maps_api_key, gpt_api_key)

Fetching device coordinates...
Device coordinates: Latitude = 32.8242, Longitude = -96.7507
Finding nearby places...
Searching places near Latitude: 32.8242, Longitude: -96.7507 within 5000 meters
Found 20 places.
Analyzing places with GPT-4o and adding full addresses...
Launching interactive graph app...


In [71]:
!pip show dash
# !pip show dash-bootstrap-components


Name: dash
Version: 2.18.2
Summary: A Python framework for building reactive web-apps. Developed by Plotly.
Home-page: https://plotly.com/dash
Author: Chris Parmer
Author-email: chris@plotly.com
License: MIT
Location: /Users/satyammittal/anaconda3/lib/python3.10/site-packages
Requires: dash-core-components, dash-html-components, dash-table, Flask, importlib-metadata, nest-asyncio, plotly, requests, retrying, setuptools, typing-extensions, Werkzeug
Required-by: dash-bootstrap-components
